In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# !pip install -qU langchain-community faiss-cpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# Get the embedding dimension (by embedding a dummy query)
dimension = len(embeddings.embed_query("dummy"))

# Create an empty FAISS index (commonly IndexFlatL2 for L2 distance; use IndexFlatIP for inner product if needed)
index = faiss.IndexFlatL2(dimension)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [10]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

In [12]:
uuids, documents

(['3601fc4f-c631-4c1b-813a-3d8be2e8df04',
  'a6aeae27-c1c7-4909-ba2d-8fb108d46679',
  'dfda47a0-1919-4a1a-a7e5-28b3a4daa16d',
  '0557edff-fe91-43a3-8bc4-f6d53a179929',
  '042d9762-fa53-426d-9005-d3a56c49700a',
  '733ba657-bbd9-4a6f-95ef-704e58a4d2fb',
  'e246ae95-6ebd-4fc3-84aa-a8ab87e8f043',
  'b5108a45-9829-44f6-8a90-e95f46806347',
  '83461be3-8269-4836-af11-3dfe7851aeec',
  '8860b4a5-f720-4f4a-82e6-f36ecee4218c'],
 [Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
  Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
  Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
  Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
  Document(metadata={'source': 'tweet'}, page_content="

In [13]:
vector_store.add_documents(documents=documents, ids=uuids)

['3601fc4f-c631-4c1b-813a-3d8be2e8df04',
 'a6aeae27-c1c7-4909-ba2d-8fb108d46679',
 'dfda47a0-1919-4a1a-a7e5-28b3a4daa16d',
 '0557edff-fe91-43a3-8bc4-f6d53a179929',
 '042d9762-fa53-426d-9005-d3a56c49700a',
 '733ba657-bbd9-4a6f-95ef-704e58a4d2fb',
 'e246ae95-6ebd-4fc3-84aa-a8ab87e8f043',
 'b5108a45-9829-44f6-8a90-e95f46806347',
 '83461be3-8269-4836-af11-3dfe7851aeec',
 '8860b4a5-f720-4f4a-82e6-f36ecee4218c']

In [33]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} -- [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! -- [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! -- [{'source': 'tweet'}]


In [38]:
results = vector_store.similarity_search_with_score(
    query="movie",
    k=1
)
for res in results:
    print(f"* {res[0].page_content} -- Score : {res[1]}")

* Wow! That was an amazing movie. I can't wait to see it again. -- Score : 1.2352288961410522


In [48]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.916558] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [49]:
try:
    vector_store.delete(ids=["8860b4a5-f720-4f4a-82e6-f36ecee4218c"])
except Exception as e:
    print(e)

Some specified ids do not exist in the current store. Ids not found: {'8860b4a5-f720-4f4a-82e6-f36ecee4218c'}


In [57]:
# 1. Get all documents (page_content + metadata)
docs = list(vector_store.docstore._dict.values())  # List[Document]
print(f"All Documents : {docs} \n")

# 2. Get all embeddings (vectors) from the FAISS index
import numpy as np
n_vectors = vector_store.index.ntotal  # Number of vectors
embeddings = np.zeros((n_vectors, vector_store.index.d), dtype="float32")
vector_store.index.reconstruct_n(0, n_vectors, embeddings)  # Fills the array
embeddings = embeddings.tolist()  # Or keep as np.array
print(f"Embeddings : {embeddings} \n")

# 3. Get IDs (mapping from index position to docstore ID)
index_to_id = {i: doc_id for i, doc_id in vector_store.index_to_docstore_id.items()}
print(f"Document Indexes : {index_to_id} \n")


# Optional: Align everything into a structured list/dict
all_data = []
for i in range(n_vectors):
    doc_id = vector_store.index_to_docstore_id.get(i)
    if doc_id:
        doc = vector_store.docstore.search(doc_id)
        all_data.append({
            "id": doc_id,
            "document": doc.page_content,
            "metadata": doc.metadata,
            # "embedding": embeddings[i]
        })
display(all_data)

All Documents : [Document(id='3601fc4f-c631-4c1b-813a-3d8be2e8df04', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'), Document(id='a6aeae27-c1c7-4909-ba2d-8fb108d46679', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'), Document(id='dfda47a0-1919-4a1a-a7e5-28b3a4daa16d', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'), Document(id='0557edff-fe91-43a3-8bc4-f6d53a179929', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'), Document(id='042d9762-fa53-426d-9005-d3a56c49700a', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."), Document(id='733ba657-bbd9-4a6f-95ef-704e58a4d2fb', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? R

[{'id': '3601fc4f-c631-4c1b-813a-3d8be2e8df04',
  'document': 'I had chocolate chip pancakes and scrambled eggs for breakfast this morning.',
  'metadata': {'source': 'tweet'}},
 {'id': 'a6aeae27-c1c7-4909-ba2d-8fb108d46679',
  'document': 'The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.',
  'metadata': {'source': 'news'}},
 {'id': 'dfda47a0-1919-4a1a-a7e5-28b3a4daa16d',
  'document': 'Building an exciting new project with LangChain - come check it out!',
  'metadata': {'source': 'tweet'}},
 {'id': '0557edff-fe91-43a3-8bc4-f6d53a179929',
  'document': 'Robbers broke into the city bank and stole $1 million in cash.',
  'metadata': {'source': 'news'}},
 {'id': '042d9762-fa53-426d-9005-d3a56c49700a',
  'document': "Wow! That was an amazing movie. I can't wait to see it again.",
  'metadata': {'source': 'tweet'}},
 {'id': '733ba657-bbd9-4a6f-95ef-704e58a4d2fb',
  'document': 'Is the new iPhone worth the price? Read this review to find out.',
  'metadata